In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Load Modules

- Kaggle 압축파일, 경로가 상이하기 때문에 Kaggle에 맞춰서 변경

In [2]:
import random, shutil, zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
base_dir = '/kaggle/working/data/sample'
os.makedirs(f'{base_dir}/cat', exist_ok=True)
os.makedirs(f'{base_dir}/dog', exist_ok=True)

In [4]:
# 현재 내 노트북 아래 폴더 확인
os.listdir('./data/sample/dog')

['dog.10002.jpg',
 'dog.1002.jpg',
 'dog.10023.jpg',
 'dog.10035.jpg',
 'dog.10037.jpg',
 'dog.10043.jpg',
 'dog.10053.jpg',
 'dog.10057.jpg',
 'dog.10064.jpg',
 'dog.10100.jpg',
 'dog.10110.jpg',
 'dog.10150.jpg',
 'dog.10152.jpg',
 'dog.10185.jpg',
 'dog.1019.jpg',
 'dog.10202.jpg',
 'dog.10204.jpg',
 'dog.10217.jpg',
 'dog.10218.jpg',
 'dog.10219.jpg',
 'dog.10243.jpg',
 'dog.10263.jpg',
 'dog.10277.jpg',
 'dog.10298.jpg',
 'dog.10299.jpg',
 'dog.1030.jpg',
 'dog.10300.jpg',
 'dog.10313.jpg',
 'dog.10334.jpg',
 'dog.10336.jpg',
 'dog.1035.jpg',
 'dog.10363.jpg',
 'dog.1037.jpg',
 'dog.10378.jpg',
 'dog.10391.jpg',
 'dog.10393.jpg',
 'dog.10403.jpg',
 'dog.10405.jpg',
 'dog.10406.jpg',
 'dog.10407.jpg',
 'dog.10413.jpg',
 'dog.10419.jpg',
 'dog.10426.jpg',
 'dog.10430.jpg',
 'dog.1044.jpg',
 'dog.10440.jpg',
 'dog.10442.jpg',
 'dog.10459.jpg',
 'dog.10461.jpg',
 'dog.10462.jpg',
 'dog.10465.jpg',
 'dog.10471.jpg',
 'dog.10476.jpg',
 'dog.10477.jpg',
 'dog.10481.jpg',
 'dog.10507.jpg'

#### 폴더 압축파일 해제
- /kaggle/input/dogs-vs-cats/test1.zip
- /kaggle/input/dogs-vs-cats/train.zip

In [5]:
zip_path = '/kaggle/input/dogs-vs-cats/train.zip'
target_path = '/kaggle/working/train'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_path)

print('done with extraction')

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/dogs-vs-cats/train.zip'

In [ ]:
## test1.zip 
zip_path = '/kaggle/input/dogs-vs-cats/test1.zip'
target_path = '/kaggle/working'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_path)

print('done with extraction')

In [ ]:
target_path

In [ ]:
image_files = os.listdir(target_path)
len(image_files)

In [ ]:
len( os.listdir(target_path + '/train'))

In [ ]:
len(os.listdir('/kaggle/working/test1'))

In [ ]:
len(os.listdir('./data/sample/dog'))

In [ ]:
len(os.listdir('./data/sample/cat'))

### 여기서 부터는 로컬 작업과 일치

In [ ]:
# 500개씩 샘플링 함수
def sample_data(src_dir, dst_dir, num_samples=500):
    os.makedirs(dst_dir, exist_ok=True)  # 폴더 생성, exist_ok=True 이미 존재하면 다시 만들지 않음
    all_files = [f for f in os.listdir(src_dir) if f.startswith(dst_dir.split('/')[-1])]
    samples = random.sample(all_files, num_samples)
    for f in samples:
        shutil.copy(os.path.join(src_dir, f), os.path.join(dst_dir, f))

#### 아래 경로만 캐글에 맞게 변경할 것

In [ ]:
# sample_data() 함수 사용
sample_data(target_path + '/train', './data/sample/cat', 500)
sample_data(target_path + '/train', './data/sample/dog', 500)

In [ ]:
# 파이토치 모듈 로드
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

In [ ]:
train_dataset = ImageFolder('./data/sample', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
len(train_loader)

In [ ]:
# 배치하나 꺼내기
images, labels = next(iter(train_loader))

In [ ]:
images[0].shape

In [ ]:
class_names = ['cat', 'dog']

In [ ]:
# 시각화
plt.figure(figsize=(12, 4))
for i in range(16): # 32개 중 반만 표현
    plt.subplot(2, 8, i+1)

    img = images[i].permute(1, 2, 0)  # 컬러채널이기 때문에 matplotlib 표현시 순서 변경
    plt.imshow(img)
    plt.title(f'{class_names[labels[i].item()]}')
    plt.axis('off')

plt.tight_layout()
plt.show()

### CNN Model

In [ ]:
# 사용모듈 로드
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 클래스 정의
class CatDogCNN(nn.Module):
    def __init__(self):
        super(CatDogCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)  # conv1, conv2 모두 사용
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128) # MaxPool(128) -> MaxPool(64) -> 32x32 
        self.fc2 = nn.Linear(128, 1)  # 마지막 분류가 0, 1 

    def forward(self, x):
        # conv -> activation -> maxpooling 2회실시
        # dense1, 2 통과
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))        
        x = x.view(-1, 64 * 32 * 32)                # flattern -> 1차원 배열화, 65,536개 입력
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))              # F.sigmoid() deprecated
        return x

### Train

In [ ]:
# cuda 준비
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CatDogCNN().to(device)
criterion = nn.BCELoss()  # Binary Cross Entropy 손실함수
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
device

In [ ]:
# 훈련함수 정의
def train(model, loader, criterion, optimizer):
    model.train()  # 훈련모드
    total_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.float().to(device).unsqueeze(1)  # 1, 0 -> [1,], [0,]
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()        
        total_loss += loss.item()

    return total_loss / len(loader)

In [ ]:
# 훈련
EPOCH = 10
for epoch in range(EPOCH):
    loss = train(model, train_loader, criterion, optimizer)
    print(f'[{epoch+1}/{EPOCH}] Loss: {loss:.4f}')

In [ ]:
# 검증
model.eval()
images, labels = next(iter(train_loader))
images = images.to(device) 
with torch.no_grad():
    outputs = model(images)
    preds = (outputs > 0.5).int().cpu().squeeze()

plt.figure(figsize=(12, 4))
for i in range(16):
    plt.subplot(2, 8, i+1)
    plt.imshow(images[i].cpu().permute(1, 2, 0))
    pred_label = 'dog' if preds[i] == 1 else 'cat'
    true_label = 'dog' if labels[i] == 1 else 'cat'
    color = 'green' if pred_label == true_label else 'red'
    plt.title(f'Pred : {pred_label}\nTrue : {true_label}', color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

#### 캐글 경로 주의!

In [ ]:
test_dir = '/kaggle/working/test1'
image_files = [f for f in os.listdir(test_dir) if f.endswith(('.jpg', '.png'))]
len(image_files)

In [ ]:
# 예측과 시각화 동시
plt.figure(figsize=(12, 12))
for i, fname in enumerate(image_files[0:16]):  # 최대 16장까지 출력 0:16, 16:32, 32:48
    img_path = os.path.join(test_dir, fname)
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img_tensor)
        pred = 'Dog' if output.item() > 0.5 else 'Cat'

    # 이미지 출력
    plt.subplot(4, 4, i + 1)
    plt.imshow(img)
    plt.title(f'{pred}\n({fname})', fontsize=10)
    plt.axis('off')

plt.tight_layout()
plt.show()

### making Submission.csv 

- 캐글에 맞춰서 추가 작성할 것

In [ ]:
test_dir = '/kaggle/working/test1'

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, path, transform):
        self.path = path
        self.files = sorted([f for f in os.listdir(path) if f.endswith('.jpg')],
                            key=lambda x: int(x.split('.')[0]))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_name = self.files[idx]
        img_path = os.path.join(self.path, img_name)
        image = Image.open(img_path).convert('RGB')
        image = self.transform(image)
        img_id = int(img_name.split('.')[0])
        return image, img_id

In [ ]:
test_dataset = TestDataset(test_dir, test_transform)

In [ ]:
len(test_dataset)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
len(test_loader)

In [ ]:
model.eval()
results = []

with torch.no_grad():
    for images, img_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).int().squeeze().cpu().numpy()
        for img_id, pred in zip(img_ids, preds):
            results.append((img_id, pred))

In [ ]:
len(results)

In [ ]:
type(results[0][0])

### results to csv

- submission.csv 파일을 submit 할것!

In [ ]:
import pandas as pd

In [ ]:
results.sort(key=lambda x: x[0])

In [ ]:
results = [(int(i), int(label)) for i, label in results]

In [ ]:
df = pd.DataFrame(results, columns=['id', 'label'])
df.tail()

In [ ]:
df.to_csv('submission.csv', index=False)

In [ ]:
df2 = pd.read_csv('submission.csv')
df2.tail()